# MaxCut

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from qiskit_optimization.applications import Maxcut

In [ ]:
n=4 # Number of nodes in graph
G=nx.Graph()
G.add_nodes_from(np.arange(0,n,1))
elist=[(0,1,1.0),(0,2,1.0),(2,3,1.0),(1,2,1.0)] # ,(0,2,1.0)
# tuple is (i,j,weight) where (i,j) is the edge
G.add_weighted_edges_from(elist)
colors = ['r' for node in G.nodes()]
pos = nx.spring_layout(G)

In [ ]:
def draw_graph(G, colors, pos):
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)

draw_graph(G, colors, pos)

In [ ]:
w = np.zeros([n,n])
for i in range(n):
    for j in range(n):
        temp = G.get_edge_data(i,j,default=0)
        if temp != 0:
            w[i,j] = temp['weight']
print(w)

In [ ]:
max_cut = Maxcut(w)
qp = max_cut.to_quadratic_program()
print(qp)

In [ ]:
from qiskit.algorithms.minimum_eigen_solvers import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms.minimum_eigensolvers import SamplingVQE
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Sampler
from qiskit.algorithms.minimum_eigensolvers import QAOA
from qiskit import transpile

In [ ]:
qubitOp, offset = qp.to_ising()
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())
result = exact.solve(qp)
result

In [ ]:
optimizer = SPSA(maxiter=300)
ry = TwoLocal(qubitOp.num_qubits, "ry", "cz", reps=1, entanglement="linear")
vqe = SamplingVQE(sampler=Sampler(), ansatz=ry, optimizer=optimizer)

# run SamplingVQE
result = vqe.compute_minimum_eigenvalue(qubitOp)
result

In [ ]:
qaoa = QAOA(optimizer=SPSA(), sampler=Sampler())
res = qaoa.compute_minimum_eigenvalue(qubitOp)
print(res)

In [ ]:
qc_transpiled = transpile(qaoa.ansatz, basis_gates=["cx", "sx", "x", "rz"])
qc_transpiled.draw(output="mpl", fold=-1)

# Add Placeholder

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit import Gate

In [ ]:
p1 = Parameter('a')
p2 = Parameter('b')

In [ ]:
unknown_edges = Gate(name='unknown_edges', num_qubits=3, params=[])

In [ ]:
decomp_qc = QuantumCircuit(3)
decomp_qc.id([0,1,2])

unknown_edges.add_decomposition(decomp_qc)

In [ ]:
from qiskit.circuit import Instruction
from qiskit.circuit import CircuitInstruction
from typing import Union
class Placeholder(Instruction):
    def __init__(self, num_qubits, label):
        self.name = "placeholder"
        super().__init__(self.name, num_qubits, 0, [], label = label)

    def inverse(self):
        return Placeholder(self.name, self.num_qubits)

# This function accepts a QuantumCircuit in addition to single instructions
# Note: you can modify it to accept a dictionary of label-instruction pairs.
def replace(self, placeholder_label, instruction: Union[Instruction, QuantumCircuit]):
    if isinstance(instruction, QuantumCircuit):
        instruction = instruction.to_instruction()

    self._data = [CircuitInstruction(instruction, _inst[1], _inst[2]) if _inst[0].name == 'placeholder' and _inst[0].label == placeholder_label else _inst for _inst in self._data]

QuantumCircuit.replace = replace

In [ ]:
def get_initial_qc_with_placeholder():
    qc = QuantumCircuit(4)
    qc.h([0,1,2,3])
    qc.barrier()
    qc.rzz(p1, 0,1)
    qc.rzz(p1, 0,2)

    #qc.append(unknown_edges, [1,2,3])
    qc.append(Placeholder(3, 'edges'), [1,2,3])

    qc.barrier()
    qc.rx(2*p2, [0,1,2,3])
    return qc 

## Quantum Circuit Nat Gates Creation

In [ ]:
def create_init_qc(unknown=False):
    qc = QuantumCircuit(4)
    qc.h([0,1,2,3])
    qc.barrier()
    qc.rzz(p1, 0,1)
    qc.rzz(p1, 0,2)
    qc = transpile(qc, basis_gates=["cx", "sx", "x", "rz"])

    qc_mixer=QuantumCircuit(4)
    qc_mixer.rx(2*p2, [0,1,2,3])
    qc_mixer = transpile(qc_mixer, basis_gates=["cx", "sx", "x", "rz"])
    
    if unknown:
        qc.append(Placeholder(3, 'edges'), [1,2,3])
    qc.barrier()
    qc.compose(qc_mixer, [0,1,2,3], inplace=True)
    return qc

In [ ]:
qc_nat_known = create_init_qc(False)
qc_nat_known.draw(output="mpl")

In [ ]:
qc_nat_unknown = create_init_qc(True)
qc_nat_unknown.draw(output="mpl")

## Determine good mapping layout for qc without unknown part

In [ ]:
from qiskit.providers.fake_provider import FakeManila
device = FakeManila()
coupling_map=device.configuration().coupling_map
qubit_offset = qc.num_qubits

In [ ]:
offline_mapped_qc = transpile(qc_nat_known, coupling_map=coupling_map, basis_gates=["cx", "sx", "x", "rz"], optimization_level=3)
offline_mapped_qc.draw(output="mpl")

In [ ]:
layout = offline_mapped_qc._layout.initial_layout
mapping = []
for elem in layout.get_virtual_bits():
    if elem.register.name == "ancilla":
        pass#mapping[layout.get_virtual_bits()[elem]] = elem.index + qubit_offset
    else:
        mapping.append(layout.get_virtual_bits()[elem])

mapping

# Add new gates at online time and compile to pre-defined layout

In [ ]:
from time import time

exec_times_partial= []

for _ in range(1000):
    qc_nat_unknown = create_init_qc(True)
    start = time()
    new_edges = QuantumCircuit(3)
    new_edges.rzz(p1, 0,2)
    new_edges.rzz(p2, 1,2)
    qc_nat_unknown.replace('edges', new_edges)

    mapped_online = transpile(qc_nat_unknown, coupling_map=coupling_map, basis_gates=["cx", "sx", "x", "rz", "edges"], initial_layout=mapping, layout_method="trivial", optimization_level=3)

    duration = time()-start
    exec_times_partial.append(duration)
np.mean(exec_times_partial)


# Runtime comparison

In [ ]:
qc = create_init_qc(True)
qc.draw()

In [ ]:
exec_times_full= []
for _ in range(1000):
    qc = get_initial_qc_with_placeholder()
    new_edges = QuantumCircuit(3)
    new_edges.rzz(p1, 0,2)
    new_edges.rzz(p2, 1,2)

    qc.replace('edges', new_edges)
    start=time()
    qc = transpile(qc, coupling_map=coupling_map, basis_gates=["cx", "sx", "x", "rz", "edges"], optimization_level=3)

    duration = time()-start
    exec_times_full.append(duration)
np.mean(exec_times_full)


In [ ]:
qc.draw(output="mpl")